In [1]:
import tensorflow as tf #no need to describe ;)
import numpy as np #allows array operation
import pandas as pd #we will use it to read and manipulate files and columns content

from nltk.corpus import stopwords #provides list of english stopwords
stop = stopwords.words('english')

D:\anaconda_files\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda_files\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda_files\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda_files\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a

In [2]:
tf.__version__

'1.14.0'

In [3]:
train = pd.read_csv('UTS_TT_DATA.csv')

In [4]:
train.head()

,row_number,affected_service,priority,description,title
0,13133,Other,Minor,"NTD ODU red. Rebooted in isolation, Still red....",NWAS-OTHER-*ANOM_ID36945*-ANOM_ID5775
1,17314,Intermittent signal,Minor,EU is reporting slow internet connection.\r\nP...,NWAS - Dropouts -*ANOM_ID42107*- ANOM_ID14570
2,17281,Loss of Signal,Minor,ISSUE: Red ODU\r\n\r\nService Address:|Addres...,NWAS -*ANOM_ID5433*-*ANOM_ID13853*
3,3252,Intermittent signal,Medium,Fault Symptom: Drop Outs\r\n\r\nHave you check...,NWAS || Intermittent || ANOM_ID47835 || -*ANOM...
4,10699,Loss of Signal,Minor,EU is unable to connect to the internet.\r\nPr...,NWAS||SERVICE LOSS||`ICE LOSS||-*ANOM_ID58216*...


In [5]:
train['description_lower'] = train["description"].str.lower()
train['description_no_punctiation'] = train['description_lower'].str.replace('[^\w\s]','')
train['description_no_punctiation'].head()

In [7]:

train['description_no_stopwords'] = train['description_no_punctiation'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [8]:
train['description_no_stopwords'].head()

0    ntd odu red rebooted isolation still red 0 sup...
1    eu reporting slow internet connection provisio...
2    issue red odu service addressaddress removed c...
3    fault symptom drop outs checked powercabling c...
4    eu unable connect internet provisioning nbn op...
Name: description_no_stopwords, dtype: object

In [9]:
tf_train = train

In [10]:
tf_train['description_no_stopwords'][1]

'eu reporting slow internet connection provisioning nbn optus checked issues found optus systems showing sync eu reported visible damage nbn equipment eu replaced ethernet cable wntd rg eu tested using ethernet device connected directly wntd experienced fault please investigate open orders none fault type intermittent servicedropouts diagnostic questions isolation test run yes active incidents time fault power status answer provided power behaviour answer provided led status answer provided led behaviour answer provided odu status answer provided odu behaviour answer provided unid status answer provided unid behaviour answer provided signal strengths status answer provided ports used answer provided ntd serial number answer provided cpe connected answer provided cpe mac address answer provided cpe powercycled answer provided alternate cpe tested answer provided test results test result 1 test id anom_id57221 test type loopback test test result passed'

In [14]:
tf_train['description_no_stopwords'][1].count(' ')

134

In [15]:
max_features=5000 #we set maximum number of words to 5000
maxlen=100 #and maximum sequence length to 100

In [16]:
tok = tf.keras.preprocessing.text.Tokenizer(num_words=max_features) #tokenizer step

In [17]:
tok.fit_on_texts(list(tf_train['description_no_stopwords'])) #fit to cleaned text

In [18]:
tf_train=tok.texts_to_sequences(list(tf_train['description_no_stopwords']))

In [19]:
print(type(tf_train)) #we see that the type is now list
print(len(tf_train[1])) #we see that the number of words of the sentence is decreased to 16
tf_train[1] #and this is how our sentece looks like now, exactly a sequence of integers

<class 'list'>
135


[70,
 200,
 130,
 66,
 74,
 159,
 23,
 111,
 104,
 133,
 137,
 111,
 182,
 131,
 135,
 70,
 152,
 154,
 148,
 23,
 115,
 70,
 510,
 172,
 80,
 125,
 1766,
 70,
 25,
 309,
 172,
 195,
 39,
 230,
 125,
 758,
 14,
 60,
 81,
 43,
 42,
 40,
 14,
 7,
 155,
 171,
 48,
 52,
 28,
 1,
 32,
 17,
 37,
 38,
 36,
 14,
 8,
 2,
 4,
 3,
 8,
 10,
 4,
 3,
 12,
 2,
 4,
 3,
 12,
 10,
 4,
 3,
 16,
 2,
 4,
 3,
 16,
 10,
 4,
 3,
 20,
 2,
 4,
 3,
 20,
 10,
 4,
 3,
 30,
 55,
 2,
 4,
 3,
 51,
 49,
 4,
 3,
 18,
 46,
 41,
 4,
 3,
 5,
 39,
 4,
 3,
 5,
 29,
 45,
 4,
 3,
 5,
 31,
 4,
 3,
 27,
 5,
 25,
 4,
 3,
 1,
 44,
 1,
 6,
 35,
 1,
 15,
 9,
 1,
 7,
 13,
 1,
 1,
 6,
 99]

In [20]:
tf_train=tf.keras.preprocessing.sequence.pad_sequences(tf_train, maxlen=maxlen) #let's execute pad step

In [21]:
print(len(tf_train[1]))
tf_train[1] #this is how our sentece looks like after the pad step we don't have anymore 16 words but 100 (equivalent to maxlen)

100


array([758,  14,  60,  81,  43,  42,  40,  14,   7, 155, 171,  48,  52,
        28,   1,  32,  17,  37,  38,  36,  14,   8,   2,   4,   3,   8,
        10,   4,   3,  12,   2,   4,   3,  12,  10,   4,   3,  16,   2,
         4,   3,  16,  10,   4,   3,  20,   2,   4,   3,  20,  10,   4,
         3,  30,  55,   2,   4,   3,  51,  49,   4,   3,  18,  46,  41,
         4,   3,   5,  39,   4,   3,   5,  29,  45,   4,   3,   5,  31,
         4,   3,  27,   5,  25,   4,   3,   1,  44,   1,   6,  35,   1,
        15,   9,   1,   7,  13,   1,   1,   6,  99])

In [22]:
train['description_no_stopwords'][1]

'eu reporting slow internet connection provisioning nbn optus checked issues found optus systems showing sync eu reported visible damage nbn equipment eu replaced ethernet cable wntd rg eu tested using ethernet device connected directly wntd experienced fault please investigate open orders none fault type intermittent servicedropouts diagnostic questions isolation test run yes active incidents time fault power status answer provided power behaviour answer provided led status answer provided led behaviour answer provided odu status answer provided odu behaviour answer provided unid status answer provided unid behaviour answer provided signal strengths status answer provided ports used answer provided ntd serial number answer provided cpe connected answer provided cpe mac address answer provided cpe powercycled answer provided alternate cpe tested answer provided test results test result 1 test id anom_id57221 test type loopback test test result passed'

In [23]:
tf_train = pd.DataFrame(tf_train)

In [26]:
tf_train.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,37,344,168,38,430,37,253,930,330,340,...,1,15,9,1,7,13,1,1,6,34
1,758,14,60,81,43,42,40,14,7,155,...,1,15,9,1,7,13,1,1,6,99
2,12,2,62,16,2,53,20,2,62,30,...,1,15,9,1,7,13,1,1,6,34
3,972,118,1169,50,976,17,43,42,40,14,...,1,15,9,1,7,13,1,1,6,99
4,115,60,81,43,42,40,14,7,59,65,...,1,15,9,1,7,13,1,1,6,34
